In [1]:
import os
import re
import ast
import numpy as np
import pandas as pd
from pprint import pprint
from tqdm.notebook import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel

import torch.nn as nn
from torch.optim import AdamW
from sklearn.metrics import classification_report

In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1


env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 8
BERT_SPECIES = 'roberta-base'
INPUT_CATS = ['src','rel','trg','mis']
OUTPUT_CATS = []
EPOCHS = 5
LEARNING_RATE = 1e-5
LOGGING_INTERVAL = 100

In [4]:
rawdf = {}
for split in ['train','val','test']:
    dpath = f"../../data/complete_{split}.csv"
    rawdf[split] = pd.read_csv(dpath)

In [5]:
rawdf['train'].mistype.unique()

array(['T Sisters', 'Simple', 'Negation', 'Set Based', 'Gradable'],
      dtype=object)

In [6]:
mistypes = ['T Sisters', 'Negation', 'Simple', 'Set Based', 'Hypo-Hyper', 'Gradable', 'Converses', 'Mero-Holo', 'Reverses']
mislocations = ['Target-Head', 'Target-Modifier', 'Relation-Modifier', 'Relation-Head', 'Source-Head', 'Source-Modifier']
num_classes_mistypes = 9
num_classes_mislocations = 6
def process_df(df):
    df = df[(df.mistype.notnull()) & (df.mistype!= 'Doubtful type')]
    df = df[(df.mislocation.notnull()) & (df.mislocation!= 'Doubtful location')]
    df['mistype'] = df.mistype.apply(lambda x: mistypes.index(x))
    df['mislocation'] = df.mislocation.apply(lambda x: mislocations.index(x))
    return df

In [7]:
for i in ['train','val','test']:
    rawdf[i] = process_df(rawdf[i])

In [8]:
rawdf['train']['mislocation'].unique()

array([0, 3, 2, 1, 4, 5])

In [9]:
input_cats = INPUT_CATS
output_cats = OUTPUT_CATS
include_context = True

In [10]:
cat2col = {'src':'SSpan','rel':'RSpan','trg':'TSpan','mis':'MSpan'}
col2cat = {col:cat for cat,col in cat2col.items()}

proc_data = {}
for split in ['train','val','test']:
    proc_data[split] = []
    print(split)
    for _, example in tqdm(rawdf[split].iterrows(),total=len(rawdf[split])):
        set_complete = True
        example_feats = {
            'text': None,
            'input_cols': {},
            'output_cols': {},
            'mistype': example['mistype'],
            'mislocation': example['mislocation']
        }
#         text = example['data'].strip() if include_context else example['data'].strip().split('\n')[0]
        text = example['claim'] + ' [SEP] '
#         text = example['claim'] + ' <SEP> '

        #         print(text)
        for io_type, cats in {
            'input_cols': input_cats,
            'output_cols': output_cats,
        }.items():
            for cat in cats:
                col = cat2col[cat]
                span = ast.literal_eval(example[col])
                if any([v==None for v in span]):
                    set_complete = False
                else:
                    if cat == 'mis': 
                        span[0] += len(text)
                        span[1] += len(text)
                    example_feats[io_type][col] = span
        text += example['context']
        example_feats['text'] = text
        if set_complete:
            proc_data[split].append(example_feats)

train


  0%|          | 0/6443 [00:00<?, ?it/s]

val


  0%|          | 0/806 [00:00<?, ?it/s]

test


  0%|          | 0/806 [00:00<?, ?it/s]

In [11]:
pprint(proc_data['train'][3]) 

{'input_cols': {'MSpan': [68, 81],
                'RSpan': [27, 39],
                'SSpan': [0, 26],
                'TSpan': [40, 56]},
 'mislocation': 3,
 'mistype': 2,
 'output_cols': {},
 'text': 'Jacqueline Kennedy Onassis is forgotten for her elegance. [SEP] She '
         'is remembered for her lifelong contributions to the arts and '
         'preservation of historic architecture , as well as for her style , '
         'elegance , and grace .'}


In [12]:
cat_tag_map = {
    'SSpan': ['<SRC>','</SRC>'],
    'RSpan': ['<REL>','</REL>'],
    'TSpan': ['<TRG>','</TRG>'],
    'MSpan': ['<MIS>','</MIS>'],
}

ext_data = {}
for split in ['train','val','test']:
    ext_data[split] = []
    print(split)
    for example in tqdm(proc_data[split]):
        new_text = ""
        new_text_split = []
        new_col_spans = {}          # character level
        new_col_spans_split = {}    # word level
        ptr = 0
        col_span_list = []
        for io_type in ['input_cols','output_cols']:
            for cat, span in example[io_type].items():
                col_span_list.append([io_type,cat,span])
        for io_type, cat, span in sorted(col_span_list,key=lambda x: x[2]):
            new_text += example['text'][ptr:span[0]]
            new_text_split.extend([v for v in example['text'][ptr:span[0]].strip().split(' ') if v!=''])
            ptr = span[0]
            
            cat_start, cat_start_split = len(new_text), len(new_text_split)
            
            if io_type == 'input_cols':
                tags = cat_tag_map[cat]
                string = tags[0] + ' ' + example['text'][span[0]:span[1]] + ' ' + tags[1]
                new_text += string
                new_text_split.extend(string.strip().split())
            else:
                string = example['text'][span[0]:span[1]]
                new_text += string
                new_text_split.extend([v for v in string.strip().split() if v!=''])
                
            cat_end, cat_end_split = len(new_text), len(new_text_split)
            
            new_col_spans[cat] = [cat_start,cat_end]
            new_col_spans_split[cat] = [cat_start_split, cat_end_split]
            
            ptr = span[1]

        new_text += example['text'][ptr:]
        new_text_split.extend(example['text'][ptr:].strip().split())
        
        wid2col = ['TXT' for _ in range(len(new_text_split))]
        for col, w_span in new_col_spans_split.items():
            for wid in range(*w_span):
                wid2col[wid] = col
        
        ext_data[split].append({
            'text': new_text,
            'text_split': new_text_split,
            'input_cols': {col:new_col_spans[col] for col in example['input_cols'].keys()},
            'output_cols': {col:new_col_spans[col] for col in example['output_cols'].keys()},
            'input_cols_split': {col:new_col_spans_split[col] for col in example['input_cols'].keys()},
            'output_cols_split': {col:new_col_spans_split[col] for col in example['output_cols'].keys()},
            'wid2col': wid2col,
            'mistype': example['mistype'],
            'mislocation': example['mislocation']
        })
#         print(example['text'])
#         print()
#         print(new_text)
#         print()
#         for col, span in new_output_cols.items():
#             print(col)
#             print(new_text[span[0]:span[1]])
        
#         break
#     break

train


  0%|          | 0/6440 [00:00<?, ?it/s]

val


  0%|          | 0/806 [00:00<?, ?it/s]

test


  0%|          | 0/803 [00:00<?, ?it/s]

In [13]:
ext_data['test'][136]

{'text': "<SRC> Sam Walton </SRC> <REL> founded </REL> <TRG> Australia and New Zealand </TRG>. [SEP] Samuel Moore `` Sam '' Walton -LRB- March 29 , 1918 -- April 5 , 1992 -RRB- was an American businessman and entrepreneur best known for founding <MIS> the retailers Walmart and Sam 's Club </MIS> .",
 'text_split': ['<SRC>',
  'Sam',
  'Walton',
  '</SRC>',
  '<REL>',
  'founded',
  '</REL>',
  '<TRG>',
  'Australia',
  'and',
  'New',
  'Zealand',
  '</TRG>',
  '.',
  '[SEP]',
  'Samuel',
  'Moore',
  '``',
  'Sam',
  "''",
  'Walton',
  '-LRB-',
  'March',
  '29',
  ',',
  '1918',
  '--',
  'April',
  '5',
  ',',
  '1992',
  '-RRB-',
  'was',
  'an',
  'American',
  'businessman',
  'and',
  'entrepreneur',
  'best',
  'known',
  'for',
  'founding',
  '<MIS>',
  'the',
  'retailers',
  'Walmart',
  'and',
  'Sam',
  "'s",
  'Club',
  '</MIS>',
  '.'],
 'input_cols': {'SSpan': [0, 23],
  'RSpan': [24, 44],
  'TSpan': [45, 83],
  'MSpan': [236, 286]},
 'output_cols': {},
 'input_cols_s

In [14]:
tokenizer = AutoTokenizer.from_pretrained(BERT_SPECIES, add_prefix_space=True)
bert_model = AutoModel.from_pretrained(BERT_SPECIES)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
print(tokenizer.all_special_tokens)
custom_spl_tokens = [tag for cat, tags in cat_tag_map.items() for tag in tags]
# custom_spl_tokens = ['<SEP>'] + [tag for cat, tags in cat_tag_map.items() for tag in tags]
print(custom_spl_tokens)
num_added_toks = tokenizer.add_tokens(custom_spl_tokens, special_tokens=True)
print(num_added_toks)
bert_model.resize_token_embeddings(len(tokenizer))
print(tokenizer.all_special_tokens)
tokenizer.save_pretrained("custom_tokenizer")

['<s>', '</s>', '<unk>', '<pad>', '<mask>']
['<SRC>', '</SRC>', '<REL>', '</REL>', '<TRG>', '</TRG>', '<MIS>', '</MIS>']
8
['<s>', '</s>', '<unk>', '<pad>', '<mask>']


('custom_tokenizer/tokenizer_config.json',
 'custom_tokenizer/special_tokens_map.json',
 'custom_tokenizer/vocab.json',
 'custom_tokenizer/merges.txt',
 'custom_tokenizer/added_tokens.json',
 'custom_tokenizer/tokenizer.json')

In [16]:
enc = {}
for split in ['train','val','test']:
    enc[split] = tokenizer.batch_encode_plus(
    [eg['text_split'] for eg in ext_data[split]],
    max_length=128,
    add_special_tokens=True,
    is_split_into_words=True,
    padding=True,
    truncation=True,
    return_tensors='pt',
    )

In [17]:
pipe_data = {} # pipeline-ready data
for split in ['train','val','test']:
    pipe_data[split] = []
    print(split)
    for i in tqdm(range(enc[split]['input_ids'].shape[0])):
        input_ids = enc[split]['input_ids'][i]
        attention_mask = enc[split]['attention_mask'][i]
        sample = ext_data[split][i]
        # output_cats = list(sample['output_cols_split'].keys())
        labels = sample['mistype']
        pipe_data[split].append({
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels,
        })

train


  0%|          | 0/6440 [00:00<?, ?it/s]

val


  0%|          | 0/806 [00:00<?, ?it/s]

test


  0%|          | 0/803 [00:00<?, ?it/s]

In [18]:
pipe_data['test'][136]

{'input_ids': tensor([    0, 50265,  1960, 14961, 50266, 50267,  4790, 50268, 50269,  1221,
             8,   188,  3324, 50270,   479,   646,  3388,   510,   742,  9581,
          3404, 45518,  1960, 12801, 14961,   111,   574, 26770,    12,   494,
          1132,  2156, 26589,   480,   587,   195,  2156,  8548,   111, 25733,
           387,    12,    21,    41,   470,  8950,     8, 11777,   275,   684,
            13, 10150, 50271,     5,  5301,  7819,     8,  1960,   128,    29,
          2009, 50272,   479,     2,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [19]:
class StringControlDataset(Dataset):
    
    def __init__(self,data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        sample = self.data[idx]
        input_ids = sample['input_ids']
        attention_mask = sample['attention_mask']
        zeros = torch.zeros(num_classes_mistypes)
        zeros[sample['labels']] = 1
        labels = zeros
        return input_ids, attention_mask, labels

In [20]:
strctrl_dataset = {}
for split in ['train','val','test']:
    strctrl_dataset[split] = StringControlDataset(pipe_data[split])

In [21]:
strctrl_dataloader = {}
for split in ['train','val','test']:
    strctrl_dataloader[split] = DataLoader(strctrl_dataset[split],
                                           batch_size=BATCH_SIZE,
#                                            shuffle=True
                                          )

In [22]:
class StringControlModel(nn.Module):
    def __init__(self,bert_model,out_classes=9):
        super().__init__()
        self.bert_model = bert_model
        self.out_classes = out_classes
        self.pre_classifier_1 = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, out_classes)

    def forward(self,input_ids,attention_mask):
        emb = self.bert_model(input_ids=input_ids,attention_mask=attention_mask)
        hidden_state_1 = emb[0]
        pooler_1 = hidden_state_1[:, 0]
        pooler_1 = self.pre_classifier_1(pooler_1)
        pooler_1 = torch.nn.Tanh()(pooler_1)
        pooler_1 = self.dropout(pooler_1)
        output = self.classifier(pooler_1)
        return output

In [23]:
model = StringControlModel(bert_model,out_classes=num_classes_mistypes)
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"trainable params: {trainable_params}")
print(f"    total params: {total_params}")
print(f"                : {round(100*trainable_params/total_params,2)}%")

model = model.to(DEVICE)

trainable params: 125249289
    total params: 125249289
                : 100.0%


In [24]:
epochs = EPOCHS
loss_fnc = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(),lr=LEARNING_RATE)
logging_interval = LOGGING_INTERVAL

In [25]:
# Training
for e in range(epochs):
    print(f'running epoch {e+1} of {epochs}')
    model.train()
    batch_loss = []
    epoch_loss = []
    for bidx, batch in tqdm(enumerate(strctrl_dataloader['train']),total=len(strctrl_dataloader['train'])):
        
        input_ids, attention_mask, labels = [v.to(DEVICE) for v in batch]
        out = model(input_ids, attention_mask)
        # labels = torch.permute(labels,(0,2,1))
        
        optimizer.zero_grad()
        loss = loss_fnc(out,labels)
        loss.backward()
        optimizer.step()
        
        epoch_loss.append(loss.item())
        batch_loss.append(loss.item())
        if len(batch_loss) > logging_interval:
            batch_loss.pop(0)
        if (bidx+1) % logging_interval == 0:
            print(f'train loss: {np.mean(batch_loss)}')
    print(f"epoch loss: {np.mean(epoch_loss)}")
    print("Running validation:")
    fin_targets=[]
    fin_outputs=[]
    batch_loss = []
    epoch_loss = []
    with torch.no_grad():
        for bidx, batch in tqdm(enumerate(strctrl_dataloader['val']),total=len(strctrl_dataloader['val'])):
            input_ids, attention_mask, labels = [v.to(DEVICE) for v in batch]
            out = model(input_ids, attention_mask)
            loss = loss_fnc(out,labels)
            epoch_loss.append(loss.item())
            fin_targets.extend(labels.cpu().detach().numpy().tolist())
            fin_outputs.extend(out.cpu().detach().numpy().tolist())
    fin_targets = [fin_targets[i].index(max(fin_targets[i])) for i in range(len(fin_targets))]
    fin_outputs = [fin_outputs[i].index(max(fin_outputs[i])) for i in range(len(fin_outputs))]
    print(classification_report(fin_targets, fin_outputs))
#     break

running epoch 1 of 5


  0%|          | 0/805 [00:00<?, ?it/s]

train loss: 1.3748845106363297
train loss: 0.8797141230106353
train loss: 0.7824963188171387
train loss: 0.6478973817825318
train loss: 0.6092531694471837
train loss: 0.5920164319872856
train loss: 0.5579438558965921
train loss: 0.5030728959292173
epoch loss: 0.7406078514660367
Running validation:


  0%|          | 0/101 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.87      0.85      0.86       502
           1       0.76      0.93      0.83       141
           2       0.72      0.29      0.41        45
           3       0.46      0.47      0.47        70
           5       0.68      0.81      0.74        48

    accuracy                           0.80       806
   macro avg       0.70      0.67      0.66       806
weighted avg       0.80      0.80      0.79       806

running epoch 2 of 5


...

              precision    recall  f1-score   support

           0       0.91      0.90      0.91       502
           1       0.82      0.89      0.85       141
           2       0.69      0.56      0.62        45
           3       0.68      0.74      0.71        70
           5       0.74      0.77      0.76        48

    accuracy                           0.85       806
   macro avg       0.77      0.77      0.77       806
weighted avg       0.86      0.85      0.85       806

running epoch 5 of 5


  0%|          | 0/805 [00:00<?, ?it/s]

train loss: 0.1958341898303479
train loss: 0.1835497250361368
train loss: 0.18440531834028662
train loss: 0.15581248282920568
train loss: 0.21121543397195638
train loss: 0.18600922347046434
train loss: 0.18069623993709683
train loss: 0.1931843160931021
epoch loss: 0.186075781028926
Running validation:


  0%|          | 0/101 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.89      0.92      0.90       502
           1       0.83      0.84      0.84       141
           2       0.73      0.49      0.59        45
           3       0.72      0.61      0.66        70
           5       0.74      0.81      0.77        48

    accuracy                           0.85       806
   macro avg       0.78      0.74      0.75       806
weighted avg       0.84      0.85      0.84       806



In [26]:
fin_targets=[]
fin_outputs=[]
batch_loss = []
epoch_loss = []
with torch.no_grad():
    for bidx, batch in tqdm(enumerate(strctrl_dataloader['test']),total=len(strctrl_dataloader['test'])):
        input_ids, attention_mask, labels = [v.to(DEVICE) for v in batch]
        out = model(input_ids, attention_mask)
        loss = loss_fnc(out,labels)
        epoch_loss.append(loss.item())
        fin_targets.extend(labels.cpu().detach().numpy().tolist())     
        fin_outputs.extend(out.cpu().detach().numpy().tolist())
fin_targets = [fin_targets[i].index(max(fin_targets[i])) for i in range(len(fin_targets))]
fin_outputs = [fin_outputs[i].index(max(fin_outputs[i])) for i in range(len(fin_outputs))]
print(classification_report(fin_targets, fin_outputs))

  0%|          | 0/101 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.92      0.91      0.92       483
           1       0.90      0.88      0.89       161
           2       0.58      0.50      0.54        42
           3       0.76      0.73      0.75        64
           5       0.68      0.87      0.76        53

    accuracy                           0.87       803
   macro avg       0.77      0.78      0.77       803
weighted avg       0.87      0.87      0.87       803

